### Google VM Instance

- google VM 인스턴스에 텐서플로우, 주피터를 설치하고 ssh, ip를 통해 접근하는 것을 정리했다.

0. Google Cloud SDK를 [설치](https://dl.google.com/dl/cloudsdk/channels/rapid/GoogleCloudSDKInstaller.exe)한다. 윈도우의 경우 명령어 창에서 gcloud 실행이
안 되기 때문에 따로 설치가 필요하다.
1. GPU에 대한 할당량 증가를 요청한다. (메일이 온다.)
2. GPU 인스턴스를 생성한다. 용량을 20GB로 설정한다.
3. SSH Public Key를 입력한다. SSH Key는 ssh-keygen으로 cmd창에서 만들 수 있고 보관은 /home/.ssh/에 보관된다.
4. CUDA를 설치한다. (*최신 버전이 9.1인데 에러가 발생해서 일단 8.0으로 설치.*) <br>
  `curl -O http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_8.0.61-1_amd64.deb` <br>
  `sudo dpkg -i ./cuda-repo-ubuntu1604_8.0.61-1_amd64.deb` <br>
  `sudo apt-get update` <br>
  `sudo apt-get install cuda-8-0` <br>
  `sudo reboot`
5. 환경 설정을 해주자. <br>
  `echo 'export CUDA_HOME=/usr/local/cuda' >> ~/.bashrc` <br>
  `echo 'export PATH=$PATH:$CUDA_HOME/bin' >> ~/.bashrc` <br>
  `echo 'export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CUDA_HOME/lib64' >> ~/.bashrc` <br>
  `source ~/.bashrc` <br>
5. cuDNN을 다운 받고 VM으로 업로드한다. 다운로드는 이 nvidia [사이트](https://developer.nvidia.com/rdp/cudnn-download)에서 받으면 되고 cmd창에 `gcloud compute scp cudnn-8.0-linux-x64-v7.tgz instance-1:/home/whikwon/`를 입력하면 된다.
6. cuDNN 파일의 압축을 풀고 복사해서 원하는 경로에 붙여넣자. 버전은 상관없으나 libcudnn관련 [에러](https://github.com/tensorflow/tensorflow/issues/12416)가 발생하면 `/usr/local/cuda/lib64` 경로에 `ln -s libcudnn.so.7.0.5 libcudnn.so.6`을 입력해서 libcudnn.so.6 파일을 만들어줘야 한다. 그리고 아래를 입력해서 압축을 풀고 원하는 경로에 파일을 옮겨주자.<br>
  `tar xzvf cudnn-8.0-linux-x64-v7.tgz` <br>
  `sudo cp cuda/lib64/* /usr/local/cuda/lib64/` <br>
  `sudo cp cuda/include/cudnn.h /usr/local/cuda/include/` <br>
  `rm -rf ~/cuda` <br>
  `rm cudnn-8.0-linux-x64-v7.tgz` <br>
7. Python3 버전을 사용할 것이므로 python으로 불러오는 default 버전을 변경해주자. ~/.bashrc 파일에 들어가서 `alias python=python3` 을 입력해주고
   cmd창에 `source ~/.bashrc`를 실행시키면 된다.
8. Anaconda를 다운로드하자. <br> `curl -O https://repo.continuum.io/archive/Anaconda3-5.0.1-Linux-x86_64.sh`
9. Anaconda를 설치하자. <br> `bash Anaconda3-5.0.1-Linux-x86_64.sh`
10. pip를 통해서 tensorflow를 설치하자. `pip install tensorflow-gpu==1.3` (*최신 버전이 1.4인데 12.22 기준 [Error](https://github.com/tensorflow/tensorflow/issues/14182) 발생.*)
11. jupyter의 설정을 변경해주자. `jupyter notebook --generate-config`를 실행한 뒤에 `/home/whikwon/.jupyter/jupyter_notebook_config.py` 경로의 파일의 맨 윗 단에 아래 내용을 입력해주자. <br>
  `c = get_config()` <br>  
  `c.IPKernelApp.pylab = 'inline'` <br>
  `c.NotebookApp.open_browser = False` <br>
  `c.NotebookApp.token = ''` <br>
  `c.NotebookApp.port = 5000` <br>
12. `gcloud compute ssh instance-1 -- -L 2222:localhost:5000` 을 실행하면 ssh를 통해 local에서 VM에 접근할 수 있다. (브라우저 창에 `localhost:2222`)
13. google cloud에서 VPC 네트워크 - 방화벽 규칙에서 tcp:5000에 대한 내용을 추가하고 인스턴스를 외부에서 접근을 허용하면 [외부 IP]:5000 의 주소로
브라우저를 통해 접근할 수 있다.

참고 사이트: <br>
https://medium.com/google-cloud/running-jupyter-notebooks-on-gpu-on-google-cloud-d44f57d22dbd <br>
https://towardsdatascience.com/running-jupyter-notebook-in-google-cloud-platform-in-15-min-61e16da34d52 <br>
https://stackoverflow.com/questions/28409548/simply-uploading-a-file-to-google-compute <br>
https://hackernoon.com/launch-a-gpu-backed-google-compute-engine-instance-and-setup-tensorflow-keras-and-jupyter-902369ed5272
